In [4]:
import os
len(os.listdir('./content/gdrive/My Drive/Indcars/test'))

120

In [5]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models

from tqdm import tqdm
import os
import PIL.Image as Image
from IPython.display import display,FileLink
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
data_dir = './content/gdrive/My Drive/Indcars'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'


In [7]:
#add transforms and load data
train_transforms = transforms.Compose([transforms.Resize((244,244)),
                                       transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


test_transforms = transforms.Compose([transforms.Resize((244,244)),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True,num_workers = 2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True , num_workers = 2)



In [8]:
def train_model(model, criterion, optimizer, scheduler, n_epochs):
    
    losses = []
    accuracies = []
    test_accuracies = []
    use_gpu = False
    
    # Check to see whether GPU is available
    if torch.cuda.is_available():
        use_gpu = True
        model.cuda()
    else:
        model.cpu()
    print(use_gpu)
    

    model.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        running_correct = 0.0
        for i, data in tqdm(enumerate(trainloader, 0)):

            inputs, labels = data
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()
            
                
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        
        epoch_loss = running_loss/len(trainloader)
        epoch_acc = 100/32*running_correct/len(trainloader)
        print("Epoch %s, loss: %.4f, acc: %.4f" % (epoch+1,epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        
        # switch the model to eval mode to evaluate on test data
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
    return model, losses, accuracies, test_accuracies

    

In [9]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            #images = images.to(device).half() # uncomment for half precision model
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc

In [10]:
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 120)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [11]:
model_trained, training_losses, training_accs, test_accs = train_model(model, criterion, optimizer, lrscheduler, n_epochs=20)

True


221it [01:02,  3.54it/s]

Epoch 1, loss: 4.7261, acc: 2.7291


Accuracy of the network on the test images: 7 %


221it [01:02,  3.55it/s]

...

Epoch 19, loss: 1.0243, acc: 78.3512


Accuracy of the network on the test images: 57 %


221it [01:03,  3.50it/s]

Epoch 20, loss: 1.0293, acc: 78.5492


Accuracy of the network on the test images: 58 %


In [12]:
model.cpu()
torch.save({'arch': 'resnet34',
            'state_dict': model.state_dict(), # Holds all the weights and biases
            },
            'indresnet34classifier.pth')

In [ ]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
classes, c_to_idx = find_classes(data_dir+"/train")

In [ ]:
# test the model on random images


# switch the model to evaluation mode to make dropout and batch norm work in eval mode
model.eval()

# transforms for the input image
loader =  transforms.Compose([transforms.Resize((244,244)),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
image = Image.open('./content/gdrive/My Drive/Indcars/test/Maruti Suzuki Omni/64.jpg')
image = loader(image).float()
image = torch.autograd.Variable(image, requires_grad=True)
image = image.unsqueeze(0)
image = image.cuda()
output = torch.exp(model.forward(image))
top_probs, top_labs = output.topk(5)

#conf, predicted = torch.ma(output.data, 1)

In [ ]:
x = 0
for labs in top_labs[0]:
    print(f"Car: {classes[int(labs)]}")
    break
    x += 1
